In [ ]:
Ex_專頁儀表板(20200722現在網頁無資料)

In [ ]:
# 台灣政治人物 Facebook 粉絲專頁清單
最近想要做Facebook上有關政治的社群網絡分析，所以寫了一個 Facebook粉絲專頁的網絡爬蟲程式。但是當我寫完程式後又出現了下個問題，就是「政治人物有這麽多，要怎麼收集這些人的粉絲頁連結清單呢?」

In [ ]:
確認目標網站
經過一番搜索後，我發現 FB專頁儀表板 這個網站已經幫我們整理好了這些資料，上面共有 742 位政治人物粉絲專頁的名稱、連結、討論人數、變化趨勢與標籤，因此我們就不用從頭開始手動整理資料囉!
http://page.board.tw/rank.php?tagid=24
那麼下一個問題就是我們要怎麼透過網路爬蟲把表格中的資料抓下來囉!

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
url = 'http://page.board.tw/rank.php?tagid=24'
resp = requests.get(url)
soup = BeautifulSoup(resp.text)
soup

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title> 政治人物  排行榜 - 專頁儀表板</title>
<meta content="website" property="og:type"/><meta content=" 政治人物  排行榜" property="og:description"/><meta content=" 政治人物  排行榜 專頁儀表板" property="og:title"/><meta content="http://page.board.tw/img/fb_img.jpg" property="og:image"/>
<!-- <link rel="stylesheet" href="/css/bootstrap.css"> -->
<link href="//cdnjs.cloudflare.com/ajax/libs/semantic-ui/2.2.6/semantic.min.css" rel="stylesheet" type="text/css"/>
<link href="https://cdn.datatables.net/1.10.15/css/dataTables.semanticui.min.css" rel="stylesheet"/>
<link href="/css/style.css" rel="stylesheet"/>
<link href="//maxcdn.bootstrapcdn.com/font-awesome/4.2.0/css/font-awesome.min.css" rel="stylesheet"/>
<script src="//code.jquery.com/jquery-1.12.4.js"></script>
<script src="http://page.board.tw/js/jquery.sparkline.min.js" type="text/javascript"></script>
<scrip

In [ ]:
結果發現網站的結構非常非常單純，資料也都放在 tr, td, a 等等 element 中，
因此我們只需要逐一取出 element 中的資料就可以抓到我們要的資料囉!

In [ ]:
soup.find('div', {'class':'ui container inside'}).find_all('tr')

[<tr><th>排名</th><th>圖示</th><th>名稱</th><th>談論人數<a href="http://gene.speaking.tw/2017/05/blog-post.html"><i class="idea icon"></i></a></th><th>變化</th><th>標籤</th></tr>,
 <tr><td>1</td><td><img src="https://graph.facebook.com/46251501064/picture?width=60&amp;height=60"/></td><td><a href="https://www.facebook.com/tsaiingwen/">蔡英文 Tsai Ing-wen</a></td><td align="right"><a href="/page.php?fpid=280">316450</a></td><td align="right"><div class="red text">-6.75%</div></td><td> <a class="ui teal tag label" href="?tagid=10">泛綠</a> <a class="ui teal tag label" href="?tagid=24">政治人物</a> <a class="ui teal tag label" href="?tagid=12">潛在總統候選人</a> </td></tr>,
 <tr><td>2</td><td><img src="https://graph.facebook.com/540113462988316/picture?width=60&amp;height=60"/></td><td><a href="https://www.facebook.com/sanmin.PehUi/">3Q 陳柏惟</a></td><td align="right"><a href="/page.php?fpid=9204">255905</a></td><td align="right"><div class="green text">+18.32%</div></td><td> <a class="ui teal tag label" href="?tagid=14

In [ ]:
開發爬蟲程式
因為網站的結構很單純，這個爬蟲程式非常快就寫完了，具體語法如下

In [ ]:
df = []
for i in soup.find('div', {'class':'ui container inside'}).find_all('tr'):
    try:
        ndf = pd.DataFrame(data = [{'名稱':i.find('a').text,
                                    '連結':i.find('a')['href'],
                                    '討論人數':i.find_all('a')[1].text,
                                    '變化':i.find('div').text,
                                    '標籤': ', '.join([i.text for i in i.find_all('a', {'class':'ui teal tag label'})])}],
                           columns = ['名稱', '連結', '討論人數', '變化', '標籤'])
        df.append(ndf)
    except:
        pass
df = pd.concat(df, ignore_index=True)
df

,名稱,連結,討論人數,變化,標籤
0,蔡英文 Tsai Ing-wen,https://www.facebook.com/tsaiingwen/,316450,-6.75%,"泛綠, 政治人物, 潛在總統候選人"
1,3Q 陳柏惟,https://www.facebook.com/sanmin.PehUi/,255905,+18.32%,"高雄市, 第三勢力, 縣市議員候選人, 政治人物"
2,韓國瑜,https://www.facebook.com/twherohan/,149779,-16.06%,"泛藍, 政治人物, 潛在總統候選人, 高雄市, 縣市長"
3,王定宇,https://www.facebook.com/wdy19690305/,131599,+34.11%,"政治人物, 泛綠, 立法委員, 台南市, 立委候選人"
4,柯文哲,https://www.facebook.com/DoctorKoWJ/,121754,-2.41%,"政治人物, 縣市長, 台北市, 潛在總統候選人, 柯文哲粉/白"
...,...,...,...,...,...
737,莊貽量-左楠量起來,https://www.facebook.com/莊貽量-左楠量起來-55577913460...,10,-33.33%,"高雄市, 縣市議員候選人, 政治人物"
738,我愛暖暖 林明智,https://www.facebook.com/ilovenuannuan/,10,-74.36%,"基隆, 政治人物, 縣市議員候選人"
739,蔡世寅,https://www.facebook.com/1531301717154449/,10,-90.20%,"政治人物, 台中市"
740,陳慈慧,https://www.facebook.com/lovejikei1122/,10,+11.11%,"政治人物, 泛綠, 縣市議員候選人, 台北市"


# 保存資料

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
path = '/content/drive/My Drive/Colab Notebooks/'
os.listdir(path)
df.to_excel(path+'fanspages.xlsx')